# DDA, PID e CASC (Modelo OLD) — Guia Passo a Passo

Este notebook explica, de forma prática, a teoria por trás do gerador de passos DDA, do controlador PID e da sincronização multieixo (CASC) no MODELO OLD, reproduzido por `interactive_old_sim.py`.


## Visão Geral do Modelo OLD

- Mestre por menor progresso relativo: `master_select_strategy = 'progress'` (argmin do progresso normalizado).
- Sem prioridade explícita ao eixo sob carga: `prefer_loaded_master = False`.
- Troca de mestre imediata: `master_switch_margin_steps = 0`.
- Rampa baseada no mestre (não no pior restante): `ramp_use_worst_remaining = False`.
- Termina quando o mestre chega: `finish_all_axes = False`.
- Feed-throttle agressivo nos escravos: threshold ≈ 200 e piso ≈ 25%.
- Sem trava de sincronismo (escravos podem 'disparar' se adiantarem).


## DDA — Gerador de Passos com Acumulador Q16

A cada subamostra do timer rápido (ex.: TIM6=50 kHz), acumulamos uma fase Q16 proporcional à velocidade alvo. Quando o acumulador ≥ 1.0 (Q16), emitimos 1 passo e subtraímos 1.0.

- IncQ16 = (v_sps / Hz) * 2^16
- Loop de substeps: `acc += IncQ16; if acc >= 1.0: acc -= 1.0; steps++`

Isso é equivalente a um DDA/Bresenham contínuo. No modelo, o DDA do firmware roda em ~50 kHz e o "chefe" (PID/CASC) em 1 kHz.


In [2]:
def dda_emit_steps(v_sps: float, Ts: float, hz: float = 50_000.0, acc_q16: float = 0.0):
    q16_one = float(1 << 16)
    substeps = int(round(hz * Ts))
    inc = (v_sps / hz) * q16_one
    emitted = 0
    acc = acc_q16
    for _ in range(substeps):
        acc += inc
        if acc >= q16_one:
            acc -= q16_one
            emitted += 1
    return emitted, acc

# Exemplo rápido: 10 k steps/s, Ts=1 ms, emite ~10 passos por ms
em, acc = dda_emit_steps(10_000.0, 0.001)
em, acc


(10, 0.0)

## PID no Domínio de Passos (com K_SCALE=256)

O erro é em **passos** DDA: `err = desired_dda_steps - actual_dda_steps`. Os termos são escalonados para inteiros de firmware usando `K_SCALE=256`.

- `pterm = (Kp * err) / K_SCALE`
- `iterm = (Ki * Iacc) / K_SCALE` (com clamp)
- `dterm = (Kd * Dfilt) / K_SCALE` (derivada filtrada)
- `corr = saturate(pterm + iterm + dterm)`
- `v_adj = saturate(v_cmd_sps_ideal + corr, 0, vmax)`


In [3]:
def pid_step(err, iacc, prev_err, d_filt, kp_i, ki_i, kd_i, k_scale=256, vmax=50_000, kd_alpha_bits=8):
    # deadband opcional pode zerar err se muito pequeno (omitido aqui)
    draw = err - prev_err
    decay = 1.0 / (1 << kd_alpha_bits)
    d_filt = d_filt + (draw - d_filt) * decay
    iacc = iacc + err
    # clamp de I (use um valor alto na prática)
    iacc = max(min(iacc, 2e5), -2e5)
    pterm = (kp_i * err) / k_scale
    iterm = (ki_i * iacc) / k_scale
    dterm = (kd_i * d_filt) / k_scale
    corr = pterm + iterm + dterm
    # saturação da correção
    corr = max(min(corr, vmax), -vmax)
    return corr, iacc, d_filt, draw

# Exemplo: ganhos inteiros (após *256), erro simples
corr, iacc, dfil, draw = pid_step(err=50.0, iacc=0.0, prev_err=0.0, d_filt=0.0, kp_i=800, ki_i=40, kd_i=120)
corr


164.154052734375

## CASC — Cross-Axis Sync (OLD)

O CASC OLD escolhe o mestre pelo **menor progresso relativo**: 

- Progresso do eixo i: `prog_i = clamp(|actual_i|, 0, total_i)`
- Mestre = argmin_i (prog_i / total_i)
- Para cada eixo i, o alvo sincronizado é: 
  
  `desired_i = (prog_mestre / total_mestre) * total_i`
  
- Erro CASC: `err_i = desired_i - actual_i` (em passos DDA).

O controle aplica `v_adj_i = saturate(v_cmd_ideal_i + corr_i)` (onde `corr_i` vem do PID em passos/s).


In [4]:
def select_master_progress(progress, totals):
    m = -1; num=0.0; den=1.0
    for i,(p,t) in enumerate(zip(progress, totals)):
        if t <= 0:
            continue
        if m == -1:
            m = i; num = p; den = t
        else:
            if p*den < num*t: # p/t < num/den
                m = i; num = p; den = t
    return m, num, den

def casc_desired_for_axis(master_num, master_den, total_i):
    if master_den <= 0 or total_i <= 0:
        return 0.0
    return (master_num * total_i) / master_den

# Exemplo: mestre e desired para cada eixo
totals = [20000,16000,12000]
actual = [5000, 4500, 6000]
progress = [min(abs(a), t) for a,t in zip(actual, totals)]
m, num, den = select_master_progress(progress, totals)
desired = [casc_desired_for_axis(num, den, t) for t in totals]
m, desired


(0, [5000.0, 4000.0, 3000.0])

## Rampa — Aceleração/Desaceleração (OLD)

- Distância de frenagem (passos): `s_brake = v^2 / (2*a)`
- Passos de acel./desacel. por Ts: `steps_avail = a * Ts`

Se o mestre está próximo do fim (no OLD, usa o "restante do mestre"), começamos a reduzir `v` até parar.


In [ ]:
def ramp_step(v_now, v_target, a_sps2, Ts, rem_master):
    s_brake = (v_now*v_now)/(2*a_sps2) if a_sps2>0 else 0.0
    steps_avail = a_sps2*Ts
    if rem_master <= s_brake:
        v_next = max(v_now - steps_avail, 0.0)
    elif v_now < v_target:
        v_next = min(v_now + steps_avail, v_target)
    elif v_now > v_target:
        v_next = max(v_now - steps_avail, v_target)
    else:
        v_next = v_now
    return v_next

ramp_step(9000.0, 10000.0, 200000.0, 0.001, rem_master=500.0)


## (Opcional) Carregar o Último CSV do OLD e Ver Métricas

Se você já rodou `interactive_old_sim.py --headless --log-dir meus_logs_old --auto-analyze`, podemos olhar o último CSV.


In [ ]:
from pathlib import Path
import csv
import os

def load_last_old_csv(dirpath='../meus_logs_old'):
    p = Path(dirpath)
    if not p.exists():
        print('Diretório não encontrado:', p); return None
    files = sorted(p.glob('interactive_sim_*.csv'))
    if not files:
        print('Nenhum CSV encontrado em', p); return None
    return files[-1]

path = load_last_old_csv()
if path:
    rows=[]
    with path.open() as f:
        r=csv.DictReader(f)
        for row in r:
            rows.append({k: float(row[k]) for k in r.fieldnames})
    print('Arquivo:', path.name, 'amostras:', len(rows), 'duração:', rows[-1]['t_s'])
    # Exemplo: erro do Z em ~1.4 s (se existir)
    z14 = [rw for rw in rows if abs(rw['t_s']-1.4) < 1e-3]
    if z14:
        rw = z14[0]; print('t=1.4s: vz=', rw['vel_sps_z'], 'ez=', rw['err_steps_z'])
    else:
        print('Sem amostra exatamente em 1.4s; rode o sim para gerar um CSV.')


Diretório não encontrado: meus_logs_old


## Como Reproduzir e Comparar

- Rodar OLD (headless):
  - `python3 interactive_old_sim.py --headless --log-dir meus_logs_old --auto-analyze`
- Rodar NEW (headless):
  - `python3 interactive_sim.py --headless --log-dir meus_logs --auto-analyze`
- Abrir os CSVs e comparar picos de `err_steps_*`, `pos_final` e `stop_fraction`.
